# 📈 Análisis Temporal: Panel Maestro UPV (2020-2024)

**Objetivo:** Analizar evolución temporal de indicadores académicos
**Periodo:** 2020 → 2024 (5 años)
**Dimensiones:** Satisfacción, Abandono, Autoeficacia, Empleabilidad
**Métodos:** Series temporales, tendencias, descomposición estacional

## 1. Importar librerías y datos

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Configurar estilo
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

# Cargar datos
df = pd.read_csv('../panel_maestro_UPV_LIMPIO.csv')
print(f'Dataset shape: {df.shape}')
print(f'Periodo: {df["año"].min()} - {df["año"].max()}')
print(f'Columnas: {list(df.columns)}')

Dataset shape: (483, 16)
Periodo: 2020 - 2023
Columnas: ['CURSO', 'COD_RUCT', 'TITULACION', 'CENTRO', 'año', 'satisfaccion_alumnos', 'satisfaccion_profesores', 'diferencia_satis', 'satisfaccion_promedio', 'tasa_abandono', 'tasa_permanencia', 'autoeficacia_3_anos', 'nivel_autoeficacia', 'porcentaje_no_desempleados', 'porcentaje_desempleados', 'nivel_empleabilidad']


## 2. Preparar datos para análisis temporal

In [2]:
# Crear agregados anuales
temporal_agg = df.groupby('año')[[
    'satisfaccion_alumnos', 'satisfaccion_profesores', 'satisfaccion_promedio',
    'tasa_abandono', 'tasa_permanencia',
    'autoeficacia_3_anos',
    'porcentaje_no_desempleados', 'porcentaje_desempleados'
]].agg(['mean', 'std', 'count']).round(3)

print('\n=== AGREGADOS ANUALES ===')
print(temporal_agg)


=== AGREGADOS ANUALES ===
     satisfaccion_alumnos              satisfaccion_profesores               \
                     mean    std count                    mean    std count   
año                                                                           
2020                6.823  1.516   113                   8.551  0.702   113   
2021                7.018  1.382   117                   8.650  0.734   117   
2022                6.893  1.419   125                   8.639  0.822   125   
2023                6.952  1.491   128                   8.750  0.682   128   

     satisfaccion_promedio              tasa_abandono  ... tasa_permanencia  \
                      mean    std count          mean  ...            count   
año                                                    ...                    
2020                 7.687  0.934   113        12.907  ...              113   
2021                 7.840  0.855   117        15.580  ...              117   
2022                 7.7

## 3. Análisis de tendencias principales

In [3]:
# Calcular promedios anuales para análisis de tendencia
series_temporal = df.groupby('año')[[
    'satisfaccion_alumnos', 'satisfaccion_profesores', 'satisfaccion_promedio',
    'tasa_abandono', 'autoeficacia_3_anos', 'porcentaje_no_desempleados'
]].mean()

print('\n=== TENDENCIAS TEMPORALES ===' )
print(series_temporal)

# Calcular cambio porcentual año a año
print('\n=== CAMBIO PORCENTUAL AÑO A AÑO ===')
cambio_pct = series_temporal.pct_change() * 100
print(cambio_pct.round(2))

# Tendencia general (diff 2020-2024)
print('\n=== CAMBIO TOTAL 2020 → 2024 ===')
cambio_total = ((series_temporal.iloc[-1] - series_temporal.iloc[0]) / series_temporal.iloc[0] * 100).round(2)
print(cambio_total)


=== TENDENCIAS TEMPORALES ===
      satisfaccion_alumnos  satisfaccion_profesores  satisfaccion_promedio  \
año                                                                          
2020              6.822743                 8.550973               7.686858   
2021              7.018325                 8.650260               7.839836   
2022              6.893206                 8.638560               7.775269   
2023              6.951521                 8.749630               7.815786   

      tasa_abandono  autoeficacia_3_anos  porcentaje_no_desempleados  
año                                                                   
2020      12.906574             5.401341                   89.316807  
2021      15.580376             5.776251                   94.436870  
2022      14.461098             6.008160                   92.576246  
2023      15.454564             5.962203                   92.052078  

=== CAMBIO PORCENTUAL AÑO A AÑO ===
      satisfaccion_alumnos  satisfacc

## 4. Visualizar tendencias principales

In [4]:
# Gráfico 1: Evolución de satisfacción
fig, axes = plt.subplots(2, 2, figsize=(15, 10), dpi=300)
fig.suptitle('Tendencias Principales 2020-2024', fontsize=16, fontweight='bold', y=0.995)

# Satisfacción
ax = axes[0, 0]
series_temporal[['satisfaccion_alumnos', 'satisfaccion_profesores', 'satisfaccion_promedio']].plot(
    ax=ax, marker='o', linewidth=2, markersize=8
)
ax.set_title('Satisfacción (Alumnos/Profesores)', fontweight='bold')
ax.set_ylabel('Puntuación (0-10)')
ax.set_xlabel('Año')
ax.grid(True, alpha=0.3)
ax.legend(loc='best', fontsize=9)

# Abandono
ax = axes[0, 1]
series_temporal[['tasa_abandono']].plot(ax=ax, color='red', marker='o', linewidth=2, markersize=8)
ax.set_title('Tasa de Abandono', fontweight='bold')
ax.set_ylabel('Porcentaje (%)')
ax.set_xlabel('Año')
ax.grid(True, alpha=0.3)
ax.legend(['Abandono'], fontsize=9)

# Autoeficacia
ax = axes[1, 0]
series_temporal[['autoeficacia_3_anos']].plot(ax=ax, color='green', marker='o', linewidth=2, markersize=8)
ax.set_title('Autoeficacia (3 años)', fontweight='bold')
ax.set_ylabel('Puntuación (0-10)')
ax.set_xlabel('Año')
ax.grid(True, alpha=0.3)
ax.legend(['Autoeficacia'], fontsize=9)

# Empleabilidad
ax = axes[1, 1]
series_temporal[['porcentaje_no_desempleados']].plot(ax=ax, color='purple', marker='o', linewidth=2, markersize=8)
ax.set_title('Empleabilidad (% No Desempleados)', fontweight='bold')
ax.set_ylabel('Porcentaje (%)')
ax.set_xlabel('Año')
ax.grid(True, alpha=0.3)
ax.legend(['Empleabilidad'], fontsize=9)

plt.tight_layout()
plt.savefig('01_tendencias_principales.png', dpi=300, bbox_inches='tight')
print('✓ Gráfico guardado: 01_tendencias_principales.png')
plt.close()

✓ Gráfico guardado: 01_tendencias_principales.png


## 5. Variabilidad (desviación estándar) a lo largo del tiempo

In [5]:
# Calcular desviación estándar anual
variabilidad = df.groupby('año')[[
    'satisfaccion_alumnos', 'satisfaccion_profesores',
    'tasa_abandono', 'autoeficacia_3_anos', 'porcentaje_no_desempleados'
]].std()

# Gráfico de variabilidad
fig, ax = plt.subplots(figsize=(12, 6), dpi=300)
variabilidad.plot(ax=ax, marker='s', linewidth=2.5, markersize=8)
ax.set_title('Variabilidad (Desviación Estándar) 2020-2024', fontsize=14, fontweight='bold')
ax.set_ylabel('Desviación Estándar')
ax.set_xlabel('Año')
ax.grid(True, alpha=0.3)
ax.legend(loc='best', fontsize=10)
plt.tight_layout()
plt.savefig('02_variabilidad_temporal.png', dpi=300, bbox_inches='tight')
print('✓ Gráfico guardado: 02_variabilidad_temporal.png')
plt.close()

print('\n=== VARIABILIDAD ANUAL ===')
print(variabilidad.round(3))

✓ Gráfico guardado: 02_variabilidad_temporal.png

=== VARIABILIDAD ANUAL ===
      satisfaccion_alumnos  satisfaccion_profesores  tasa_abandono  \
año                                                                  
2020                 1.516                    0.702          9.027   
2021                 1.382                    0.734         10.356   
2022                 1.419                    0.822         11.438   
2023                 1.491                    0.682         14.271   

      autoeficacia_3_anos  porcentaje_no_desempleados  
año                                                    
2020                1.713                      15.091  
2021                1.605                       9.940  
2022                1.426                       9.877  
2023                1.560                      10.294  


## 6. Análisis por CENTRO (evolución comparada)

In [6]:
# Evolución por centro
temporal_centro = df.groupby(['año', 'CENTRO']).agg({
    'satisfaccion_promedio': 'mean',
    'tasa_abandono': 'mean',
    'autoeficacia_3_anos': 'mean',
    'porcentaje_no_desempleados': 'mean'
}).round(2)

print('\n=== INDICADORES POR CENTRO Y AÑO ===')
print(temporal_centro)

# Identificar centros
centros = df['CENTRO'].unique()
print(f'\nCentros identificados: {len(centros)}')
for i, centro in enumerate(sorted(centros), 1):
    n_prog = len(df[df['CENTRO'] == centro]['TITULACION'].unique())
    print(f'{i}. {centro} ({n_prog} programas)')


=== INDICADORES POR CENTRO Y AÑO ===
                                                         satisfaccion_promedio  \
año  CENTRO                                                                      
2020 E. POLITÉCNICA SUPERIOR DE ALCOY                                     7.95   
     E.T.S. DE ARQUITECTURA                                               7.15   
     E.T.S. DE INGENIERIA DEL DISEÑO                                      7.63   
     E.T.S. DE INGENIERÍA DE EDIFICACIÓN                                  8.21   
     E.T.S. DE INGENIERÍA INFORMÁTICA                                     7.76   
     E.T.S.I. AGRONÓMICA Y DEL MEDIO NATURAL                              6.97   
     E.T.S.I. CAMINOS, CANALES Y PUERTOS                                  7.38   
     E.T.S.I. DE TELECOMUNICACIÓN                                         7.32   
     E.T.S.I. GEODESICA, CARTOGRAFICA Y TOP.                              8.27   
     E.T.S.I. INDUSTRIALES                                  

## 7. Gráfico comparativo por CENTRO

In [7]:
# Seleccionar centros principales (por número de programas)
centros_top = df['CENTRO'].value_counts().head(5).index.tolist()
df_centros = df[df['CENTRO'].isin(centros_top)]

# Gráfico: Satisfacción por centro a lo largo del tiempo
fig, axes = plt.subplots(2, 2, figsize=(15, 10), dpi=300)
fig.suptitle('Evolución por Centros Principales 2020-2024', fontsize=16, fontweight='bold', y=0.995)

# Satisfacción
ax = axes[0, 0]
for centro in centros_top:
    data = df_centros[df_centros['CENTRO'] == centro].groupby('año')['satisfaccion_promedio'].mean()
    ax.plot(data.index, data.values, marker='o', label=centro[:20], linewidth=2)
ax.set_title('Satisfacción Promedio', fontweight='bold')
ax.set_ylabel('Puntuación')
ax.set_xlabel('Año')
ax.grid(True, alpha=0.3)
ax.legend(fontsize=8)

# Abandono
ax = axes[0, 1]
for centro in centros_top:
    data = df_centros[df_centros['CENTRO'] == centro].groupby('año')['tasa_abandono'].mean()
    ax.plot(data.index, data.values, marker='s', label=centro[:20], linewidth=2)
ax.set_title('Tasa Abandono', fontweight='bold')
ax.set_ylabel('Porcentaje')
ax.set_xlabel('Año')
ax.grid(True, alpha=0.3)
ax.legend(fontsize=8)

# Autoeficacia
ax = axes[1, 0]
for centro in centros_top:
    data = df_centros[df_centros['CENTRO'] == centro].groupby('año')['autoeficacia_3_anos'].mean()
    ax.plot(data.index, data.values, marker='^', label=centro[:20], linewidth=2)
ax.set_title('Autoeficacia', fontweight='bold')
ax.set_ylabel('Puntuación')
ax.set_xlabel('Año')
ax.grid(True, alpha=0.3)
ax.legend(fontsize=8)

# Empleabilidad
ax = axes[1, 1]
for centro in centros_top:
    data = df_centros[df_centros['CENTRO'] == centro].groupby('año')['porcentaje_no_desempleados'].mean()
    ax.plot(data.index, data.values, marker='D', label=centro[:20], linewidth=2)
ax.set_title('Empleabilidad', fontweight='bold')
ax.set_ylabel('Porcentaje')
ax.set_xlabel('Año')
ax.grid(True, alpha=0.3)
ax.legend(fontsize=8)

plt.tight_layout()
plt.savefig('03_evolucion_por_centros.png', dpi=300, bbox_inches='tight')
print('✓ Gráfico guardado: 03_evolucion_por_centros.png')
plt.close()

✓ Gráfico guardado: 03_evolucion_por_centros.png


## 8. Análisis de volatilidad y cambios

In [8]:
# Calcular cambios año a año
cambios_yoy = series_temporal.diff()

fig, ax = plt.subplots(figsize=(12, 6), dpi=300)
cambios_yoy.plot(kind='bar', ax=ax, width=0.8)
ax.set_title('Cambios Año a Año (Diferencia Absoluta)', fontsize=14, fontweight='bold')
ax.set_ylabel('Cambio')
ax.set_xlabel('Período')
ax.grid(True, alpha=0.3, axis='y')
ax.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax.legend(loc='best', fontsize=9)
plt.xticks(rotation=0)
plt.tight_layout()
plt.savefig('04_cambios_ano_a_ano.png', dpi=300, bbox_inches='tight')
print('✓ Gráfico guardado: 04_cambios_ano_a_ano.png')
plt.close()

print('\n=== CAMBIOS AÑO A AÑO ===')
print(cambios_yoy.round(3))

✓ Gráfico guardado: 04_cambios_ano_a_ano.png

=== CAMBIOS AÑO A AÑO ===
      satisfaccion_alumnos  satisfaccion_profesores  satisfaccion_promedio  \
año                                                                          
2020                   NaN                      NaN                    NaN   
2021                 0.196                    0.099                  0.153   
2022                -0.125                   -0.012                 -0.065   
2023                 0.058                    0.111                  0.041   

      tasa_abandono  autoeficacia_3_anos  porcentaje_no_desempleados  
año                                                                   
2020            NaN                  NaN                         NaN  
2021          2.674                0.375                       5.120  
2022         -1.119                0.232                      -1.861  
2023          0.993               -0.046                      -0.524  


## 9. Correlación entre indicadores en el tiempo

In [9]:
# Correlación temporal
variables_temp = [
    'satisfaccion_promedio', 'tasa_abandono', 'autoeficacia_3_anos', 'porcentaje_no_desempleados'
]
corr_temporal = series_temporal[variables_temp].corr()

print('\n=== CORRELACIÓN ENTRE INDICADORES (SERIES ANUALES) ===')
print(corr_temporal.round(3))

# Heatmap de correlaciones
fig, ax = plt.subplots(figsize=(8, 6), dpi=300)
sns.heatmap(corr_temporal, annot=True, fmt='.3f', cmap='RdBu_r', center=0,
            square=True, linewidths=1, cbar_kws={'label': 'Correlación'}, ax=ax,
            vmin=-1, vmax=1)
ax.set_title('Correlación Temporal entre Indicadores', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('05_correlacion_temporal.png', dpi=300, bbox_inches='tight')
print('\n✓ Gráfico guardado: 05_correlacion_temporal.png')
plt.close()


=== CORRELACIÓN ENTRE INDICADORES (SERIES ANUALES) ===
                            satisfaccion_promedio  tasa_abandono  \
satisfaccion_promedio                       1.000          0.994   
tasa_abandono                               0.994          1.000   
autoeficacia_3_anos                         0.728          0.745   
porcentaje_no_desempleados                  0.926          0.879   

                            autoeficacia_3_anos  porcentaje_no_desempleados  
satisfaccion_promedio                     0.728                       0.926  
tasa_abandono                             0.745                       0.879  
autoeficacia_3_anos                       1.000                       0.654  
porcentaje_no_desempleados                0.654                       1.000  



✓ Gráfico guardado: 05_correlacion_temporal.png


## 10. Análisis de desempeño por TITULACION (top/bottom)

In [10]:
# Identificar mejores y peores titulaciones
perf_titulaciones = df.groupby('TITULACION').agg({
    'satisfaccion_promedio': 'mean',
    'tasa_abandono': 'mean',
    'autoeficacia_3_anos': 'mean',
    'porcentaje_no_desempleados': 'mean'
}).round(2)

# Calcular score de desempeño
perf_titulaciones['score_desempeño'] = (
    perf_titulaciones['satisfaccion_promedio'] * 0.3 +
    (100 - perf_titulaciones['tasa_abandono']) * 0.3 / 100 +
    perf_titulaciones['autoeficacia_3_anos'] * 0.2 +
    perf_titulaciones['porcentaje_no_desempleados'] * 0.2 / 100
).round(2)

perf_titulaciones_sorted = perf_titulaciones.sort_values('score_desempeño', ascending=False)

print('\n=== MEJORES TITULACIONES (Top 10) ===')
print(perf_titulaciones_sorted.head(10))

print('\n=== PEORES TITULACIONES (Bottom 10) ===')
print(perf_titulaciones_sorted.tail(10))


=== MEJORES TITULACIONES (Top 10) ===
                                                    satisfaccion_promedio  \
TITULACION                                                                  
MÁSTER UNIVERSITARIO EN INGENIERÍA DE ANÁLISIS ...                   8.47   
MÁSTER UNIVERSITARIO EN CIBERSEGURIDAD Y CIBERI...                   8.58   
MÁSTER UNIVERSITARIO EN INTELIGENCIA ARTIFICIAL...                   8.76   
MÁSTER UNIVERSITARIO EN INGENIERÍA Y TECNOLOGÍA...                   8.43   
MÁSTER UNIVERSITARIO EN GESTIÓN DE EMPRESAS, PR...                   9.50   
MÁSTER UNIVERSITARIO EN LENGUAS Y TECNOLOGÍA                         8.79   
MÁSTER UNIVERSITARIO EN SEGURIDAD NUCLEAR Y PRO...                   8.89   
MÁSTER UNIVERSITARIO EN INGENIERÍA DEL MANTENIM...                   9.39   
MÁSTER UNIVERSITARIO EN COMPUTACIÓN EN LA NUBE ...                   8.31   
MÁSTER UNIVERSITARIO EN INGENIERÍA DE SISTEMAS ...                   8.35   

                                    

## 11. Movilidad entre clusters a lo largo del tiempo

In [11]:
# Cargar clusters previos si existen, sino crear nuevos
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Variables para clustering temporal
features = ['satisfaccion_promedio', 'tasa_abandono', 'autoeficacia_3_anos', 'porcentaje_no_desempleados']

# Crear clusters para cada año
clusters_por_año = {}
for año in sorted(df['año'].unique()):
    df_año = df[df['año'] == año][features].dropna()
    
    # Estandarizar
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(df_año)
    
    # K-means
    kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
    clusters = kmeans.fit_predict(features_scaled)
    
    clusters_por_año[año] = clusters.mean()  # Valor promedio de cluster
    
    print(f'{año}: Cluster promedio = {clusters.mean():.2f}')

print('\n✓ Análisis de clusters por año completado')

2020: Cluster promedio = 0.91
2021: Cluster promedio = 1.08
2022: Cluster promedio = 0.86
2023: Cluster promedio = 0.59

✓ Análisis de clusters por año completado


## 12. Heatmap de indicadores por año

In [12]:
# Crear matriz para heatmap
heatmap_data = series_temporal[variables_temp].T

# Normalizar a escala 0-1 para mejor visualización
scaler = StandardScaler()
heatmap_normalized = pd.DataFrame(
    scaler.fit_transform(heatmap_data.T).T,
    index=heatmap_data.index,
    columns=heatmap_data.columns
)

fig, ax = plt.subplots(figsize=(10, 5), dpi=300)
sns.heatmap(heatmap_normalized, annot=series_temporal[variables_temp].T.round(2),
            fmt='g', cmap='RdYlGn', center=0, linewidths=1,
            cbar_kws={'label': 'Desviación Estándar'}, ax=ax)
ax.set_title('Evolución Temporal de Indicadores (2020-2024)', fontsize=14, fontweight='bold')
ax.set_xlabel('Año')
ax.set_ylabel('Indicador')
plt.tight_layout()
plt.savefig('06_heatmap_temporal.png', dpi=300, bbox_inches='tight')
print('✓ Gráfico guardado: 06_heatmap_temporal.png')
plt.close()

✓ Gráfico guardado: 06_heatmap_temporal.png


## 13. Exportar resultados a CSV

In [13]:
# Exportar series temporal
series_temporal.to_csv('series_temporal_principal.csv')
print('✓ Exportado: series_temporal_principal.csv')

# Exportar cambios año a año
cambios_yoy.to_csv('cambios_ano_a_ano.csv')
print('✓ Exportado: cambios_ano_a_ano.csv')

# Exportar variabilidad
variabilidad.to_csv('variabilidad_anual.csv')
print('✓ Exportado: variabilidad_anual.csv')

# Exportar correlaciones
corr_temporal.to_csv('correlaciones_temporal.csv')
print('✓ Exportado: correlaciones_temporal.csv')

# Exportar desempeño titulaciones
perf_titulaciones_sorted.to_csv('desempeño_titulaciones.csv')
print('✓ Exportado: desempeño_titulaciones.csv')

print('\n=== ANÁLISIS TEMPORAL COMPLETADO ===')
print('Archivos generados:')
print('  - 6 gráficos PNG (300 DPI)')
print('  - 5 archivos CSV')

✓ Exportado: series_temporal_principal.csv
✓ Exportado: cambios_ano_a_ano.csv
✓ Exportado: variabilidad_anual.csv
✓ Exportado: correlaciones_temporal.csv
✓ Exportado: desempeño_titulaciones.csv

=== ANÁLISIS TEMPORAL COMPLETADO ===
Archivos generados:
  - 6 gráficos PNG (300 DPI)
  - 5 archivos CSV
